In [5]:
import requests
import pandas as pd 
from datetime import datetime
import smtplib 
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os

API_KEY = ""
CITIES = ["Chennai", "Bangalore", "Mumbai", "Delhi", "Hyderabad"]

def fetch_weather(city):
    """Fetch live weather data for one city"""
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        return {
            "City": city,
            "Temperature (°C)": data["main"]["temp"],
            "Feels Like (°C)": data["main"]["feels_like"],
            "Humidity (%)": data["main"]["humidity"],
            "Weather": data["weather"][0]["description"].title(),
            "Wind Speed (m/s)": data["wind"]["speed"]
        }
    else:
        print(f"⚠️ Error fetching {city}: {data.get('message', 'Unknown error')}")
        return None


def create_report():
    all_data = []
    for city in CITIES:
        record = fetch_weather(city)
        if record:
            all_data.append(record)
    df = pd.DataFrame(all_data)
    return df

def save_report(df):
    date_str = datetime.now().strftime("%Y-%m-%d")
    file_name = f"Daily_Weather_Report_{date_str}.xlsx"
    df.to_excel(file_name, index=False)
    return file_name, date_str

def send_email_report(file_name, date_str):
    sender = "@gmail.com"
    receiver = "@exampl.com"
    password = ""  # Use App Password for Gmail

    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = receiver
    msg["Subject"] = f"🌤 Daily Weather Report - {date_str}"

    body = f"""
    Hello Team,

    Please find attached the **daily weather report** generated automatically on {date_str}.

    Cities covered: {', '.join(CITIES)}.

    Regards,  
    Weather Automation Bot 🌦
    """
    msg.attach(MIMEText(body, "plain"))


    with open(file_name, "rb") as f:
        attach = MIMEApplication(f.read(), _subtype="xlsx")
        attach.add_header("Content-Disposition", "attachment", filename=file_name)
        msg.attach(attach)

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender, password)
            server.send_message(msg)
        print("✅ Email sent successfully!")
    except Exception as e:
        print("❌ Error sending email:", e)
    finally:
        os.remove(file_name)  # cleanup temporary file


def automate_weather_report():
    print("Fetching weather data...")
    df = create_report()
    file_name, date_str = save_report(df)
    send_email_report(file_name, date_str)
    print("Automation completed!")

# Run the automation
if __name__ == "__main__":
    automate_weather_report()


Fetching weather data...
✅ Email sent successfully!
Automation completed!
